# Run mega- and meta-analysis

Steps:
1. Make a toy dataset
1. Run mega-analysis (linear mixed effects model with random intercepts for site)
1. Group dataset by site and run OLS on each site separately to construct derived toy meta-analysis dataset
1. Run meta-analysis with DerSimonian-Laird between-study variance estimator

In [1]:
import numpy as np
import statsmodels.api as sm

## Wrangle some example data

In [2]:
data = sm.datasets.anes96.load_pandas().data

In [3]:
data.head()

,popul,TVnews,selfLR,ClinLR,DoleLR,PID,age,educ,income,vote,logpopul
0,0.0,7.0,7.0,1.0,6.0,6.0,36.0,3.0,1.0,1.0,-2.302585
1,190.0,1.0,3.0,3.0,5.0,1.0,20.0,4.0,1.0,0.0,5.247550
2,31.0,7.0,2.0,2.0,6.0,1.0,24.0,6.0,1.0,0.0,3.437208
3,83.0,4.0,3.0,4.0,5.0,1.0,28.0,6.0,1.0,0.0,4.420045
4,640.0,7.0,5.0,6.0,4.0,0.0,68.0,6.0,1.0,0.0,6.461624


In [4]:
dat = data['popul']
bins = np.linspace(0, np.max(dat) + 1, 20)
digitized = np.digitize(dat, bins)
idx = {i: np.where(digitized == i)[0] for i in range(1, len(bins))}
idx = {k: v for k, v in idx.items() if v.size}

# Assign "site" based on grouped populations
data['site'] = 0
i = 0
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
for k, v in idx.items():
    data.loc[v, 'site'] = letters[i]
    i += 1

In [5]:
data.head()

,popul,TVnews,selfLR,ClinLR,DoleLR,PID,age,educ,income,vote,logpopul,site
0,0.0,7.0,7.0,1.0,6.0,6.0,36.0,3.0,1.0,1.0,-2.302585,a
1,190.0,1.0,3.0,3.0,5.0,1.0,20.0,4.0,1.0,0.0,5.247550,a
2,31.0,7.0,2.0,2.0,6.0,1.0,24.0,6.0,1.0,0.0,3.437208,a
3,83.0,4.0,3.0,4.0,5.0,1.0,28.0,6.0,1.0,0.0,4.420045,a
4,640.0,7.0,5.0,6.0,4.0,0.0,68.0,6.0,1.0,0.0,6.461624,b


## The mega-analysis

Random intercepts model using site

Do age and education predict TV news watching?

In [6]:
model = sm.MixedLM.from_formula("TVnews ~ age + educ", data, groups=data["site"])
fitted_model = model.fit()
print(fitted_model.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: TVnews    
No. Observations: 944     Method:             REML      
No. Groups:       7       Scale:              5.9387    
Min. group size:  7       Log-Likelihood:     -2189.2176
Max. group size:  833     Converged:          Yes       
Mean group size:  134.9                                 
---------------------------------------------------------
           Coef.  Std.Err.    z     P>|z|  [0.025  0.975]
---------------------------------------------------------
Intercept  0.680     0.441   1.542  0.123  -0.184   1.543
age        0.067     0.005  13.738  0.000   0.058   0.077
educ       0.012     0.051   0.234  0.815  -0.087   0.111
Group Var  0.267     0.138                               



## Create the meta-analysis dataset

In [7]:
import pandas as pd

data["intercept"] = 1
target_vars = ["educ", "age", "intercept"]

# calculate mean and variance for each variance of interest
meta_df = []
for site_name, site_df in data.groupby("site"):
    model = sm.OLS(site_df["TVnews"], site_df[target_vars])
    fitted_model = model.fit()
    
    # extract parameter estimates and errors as Series
    coefficients = fitted_model.params
    std_errors = fitted_model.bse
    
    # convert standard error to sampling variance
    sampling_variances = std_errors ** 2
    names = [n + "_var" for n in sampling_variances.index.tolist()]
    sampling_variances.index = names
    
    # combine Series and convert to DataFrame
    coefficients = coefficients.append(sampling_variances)
    coefficients["site"] = site_name
    coefficients["sample_size"] = site_df.shape[0]
    temp_df = pd.DataFrame(coefficients).T
    meta_df.append(temp_df)
# Combine across sites and convert objects to floats
meta_df = pd.concat(meta_df).reset_index(drop=True)
meta_df = meta_df.convert_dtypes()

In [8]:
meta_df.to_csv("meta_data.txt", sep='\t', index=False)
print(meta_df.to_markdown())

|    |       educ |        age |   intercept |   educ_var |     age_var |   intercept_var | site   |   sample_size |
|---:|-----------:|-----------:|------------:|-----------:|------------:|----------------:|:-------|--------------:|
|  0 | -0.0343665 |  0.0685226 |    0.621957 | 0.00285178 | 2.69295e-05 |        0.146686 | a      |           833 |
|  1 |  0.337121  |  0.0831343 |   -0.754383 | 0.0662437  | 0.000657908 |        2.64682  | b      |            42 |
|  2 | -0.131197  |  0.0794553 |   -0.551759 | 0.116955   | 0.000764189 |        5.72236  | c      |            22 |
|  3 |  0.56336   |  0.125616  |   -4.06326  | 0.0847798  | 0.000776433 |        5.00019  | d      |            12 |
|  4 |  1.47761   |  0.0410448 |   -4.48134  | 0.596322   | 0.00455964  |       12.0257   | e      |            10 |
|  5 |  0.398824  | -0.0435086 |    3.97413  | 0.472602   | 0.00383122  |       25.8563   | f      |             7 |
|  6 |  0.122459  | -0.016405  |    4.29702  | 0.191657   | 0.00

## The meta-analysis

Are age and education significant predictors of TV news watching across the literature?

In [9]:
from pymare.estimators import DerSimonianLaird
from pymare import Dataset

In /Users/tsalo/anaconda/envs/python3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /Users/tsalo/anaconda/envs/python3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /Users/tsalo/anaconda/envs/python3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [10]:
metamodel = DerSimonianLaird()
dset = Dataset(
    y=meta_df[["age", "educ"]].values, 
    v=meta_df[["age_var", "educ_var"]].values,
    add_intercept=True,
)
metamodel.fit_dataset(dset)

In [11]:
summary = metamodel.summary()
summary.get_fe_stats()

{'est': array([[0.0674741 , 0.18878075]]),
 'se': array([[0.01483012, 0.14301033]]),
 'ci_l': array([[ 0.0384076 , -0.09151434]]),
 'ci_u': array([[0.09654059, 0.46907585]]),
 'z': array([[4.54980173, 1.32004978]]),
 'p': array([[5.36964868e-06, 1.86818398e-01]])}